# Vertical Bin Packing Problem

- Airport and Cargo Operations (AE4446)
- Zach Kelly (5901405), Timotei Dudas ()
- May 5th, 2023

### Imports

In [363]:
import pandas as pd
import numpy as np 
import gurobipy as gp
from gurobipy import Model, GRB, GurobiError, quicksum, LinExpr

### Input Data

In [364]:
# load data 
ulds = pd.read_pickle('../data/B.pickle')
items = pd.read_pickle('../data/R.pickle')

print('----- ULDs -----')
display(ulds)
print('----- Items -----')
display(items)

----- ULDs -----


{0: (0, [300, 155, 2, 140, 0, 0, 0]),
 1: (0, [300, 155, 2, 140, 0, 0, 0]),
 2: (1, [192, 155, 2, 200, 0, 0, 0]),
 3: (1, [192, 155, 2, 200, 0, 0, 0])}

----- Items -----


{0: (98, 50, 1, 0, 0, 0),
 1: (89, 46, 1, 0, 0, 0),
 2: (64, 24, 1, 0, 0, 0),
 3: (93, 29, 1, 1, 0, 0),
 4: (114, 54, 1, 1, 0, 0),
 5: (95, 60, 1, 0, 0, 0),
 6: (63, 28, 1, 0, 0, 0),
 7: (100, 46, 0, 0, 0, 0),
 8: (52, 61, 1, 0, 0, 0),
 9: (45, 46, 1, 0, 0, 0),
 10: (111, 32, 1, 0, 0, 0),
 11: (109, 38, 1, 0, 1, 0),
 12: (97, 57, 1, 0, 0, 0),
 13: (51, 29, 1, 0, 0, 0),
 14: (86, 54, 1, 0, 0, 0),
 15: (114, 31, 1, 1, 0, 0),
 16: (81, 47, 1, 0, 0, 0),
 17: (78, 25, 1, 0, 0, 0),
 18: (78, 44, 1, 0, 0, 0),
 19: (68, 33, 1, 0, 0, 0),
 20: (51, 45, 1, 0, 0, 0),
 21: (84, 36, 1, 0, 0, 1),
 22: (66, 35, 0, 0, 0, 0),
 23: (68, 42, 1, 1, 0, 0),
 24: (108, 57, 1, 1, 1, 0)}

In [365]:
# toy data
ulds = {0: (0, [300, 155, 2, 140, 0, 0, 0]), 1: (1, [192, 155, 2, 200, 0, 0, 0])}

items = {0: (98, 50, 1, 0, 0, 0),
 1: (89, 46, 1, 0, 0, 0),
 2: (64, 24, 1, 0, 0, 0),
 3: (93, 29, 1, 1, 0, 0),
 4: (114, 54, 1, 1, 0, 0),
 5: (95, 60, 1, 0, 0, 0),
 6: (63, 28, 1, 0, 0, 0),
 7: (100, 46, 0, 0, 0, 0),
 8: (52, 61, 1, 0, 0, 0),
 9: (45, 46, 1, 0, 0, 0)}

### Setup Model

In [366]:
print('Creating Model')
print('--------------------')
model = Model()

Creating Model
--------------------


### Model Parameters 

In [367]:
# total number of boxes to be packed
n = len(items.keys())
# total number of available ULDs
m = len(ulds.keys())
# dimensions of items
li = [items[i][0] for i in range(n)] # length
hi = [items[i][1] for i in range(n)] # height
# dimensions of ULDs 
lc = [ulds[j][1][0] for j in range(m)]
hc = [ulds[j][1][1] for j in range(m)]
# maximum area of ULDs
Aj = [lc[j]*hc[j] for j in range(len(lc))]
# total area of items
Ai = np.sum([li[i]*hi[i] for i in range(len(li))])
# find largest uld length and width
Lj_max = np.max(np.array(list(dict(ulds.values()).values()))[:,0])
Hj_max = np.max(np.array(list(dict(ulds.values()).values()))[:,1])

### Model Variables

In [368]:
print('Creating Decision Variables')
print('--------------------')

### define decision variables ###

p_ij = {} # if item i is in ULD j 

for i in range(n):
    for j in range(m):
        p_ij[i,j]=model.addVar(lb=0, ub=1, vtype=GRB.BINARY,name="p[%s,%s]"%(i,j))

u_j = {} # if container j is used 
a_j = {} # unused area of container j 
for j in range(m):
    u_j[j] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="u[%s]"%(j))
    a_j[j] = model.addVar(lb=0, ub=Hj_max*Lj_max, vtype=GRB.CONTINUOUS, name="a[%s]"%(j))

x_i = {} # x location of lower left corner of item i
z_i = {} # z location of lower left corner of item i
x_i_ = {} # x location of upper right corner of item i
z_i_ = {} # z location of upper right corner of item i 

for i in range(n):
    x_i[i] = model.addVar(lb=0, ub=Lj_max, vtype=GRB.CONTINUOUS, name="x_[%s]"%(i))
    x_i_[i] = model.addVar(lb=0, ub=Lj_max, vtype=GRB.CONTINUOUS, name="x_[%s]_"%(i))
    z_i[i] = model.addVar(lb=0, ub=Hj_max, vtype=GRB.CONTINUOUS, name="z_[%s]"%(i))
    z_i_[i] = model.addVar(lb=0, ub=Hj_max, vtype=GRB.CONTINUOUS, name="z_[%s]_"%(i))

r_i11 = {} # if length is along x axis 
r_i13 = {} # if height is along x axis 
r_i31 = {} # if length is along z axis 
r_i33 = {} # if height is along z axis 

for i in range(n):
    r_i11[i] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="r_[%s]11"%(i))
    r_i13[i] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="r_[%s]13"%(i))
    r_i31[i] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="r_[%s]31"%(i))
    r_i33[i] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="r_[%s]33"%(i))

xp_ik = {} # if item i is to the right of item k 
xp_ki = {} # if item k is to the right of item i
zp_ik = {} # if item i is above item k 
zp_ki = {} # if item k is below item i 

for i in range(n):
    for k in range(i+1,n):
        xp_ik[i,k] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="xp_ik[%s,%s]"%(i,k))
        xp_ki[k,i] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="xp_ki[%s,%s]"%(k,i))
        zp_ik[i,k] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="zp_ik[%s,%s]"%(i,k))
        zp_ki[k,i] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="zp_ki[%s,%s]"%(k,i))

r_iab = {} # if side b of item i is along axis a 

for i in range(n):
    for a in range(0,2):
        for b in range(0,2):
            r_iab[i,a,b] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="r_iab[%s,%s,%s]"%(i,a,b))

lv = {} # if the length of the item can be in a vertical position
hv = {} # if the height of the item can be in a vertical position

for i in range(n):
    lv[i] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="lv[%s]"%(i))
    hv[i] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="hv[%s]"%(i))

g_i = {} # if item i is on the ground

for i in range(n):
    g_i[i] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="g_i[%s]"%(i))

h_ik = {} # if box k has a suitable height to support box i

for i in range(n):
    for k in range(i+1,n):
        h_ik[i,k] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="g_ik[%s,%s]"%(i,k))

o_ik = {} # if the projection of the items i and k onto the x plane have a nonempty intersection

for i in range(n):
    for k in range(i+1,n):
        o_ik[i,k] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="o_ik[%s,%s]"%(i,k))

s_ik = {} # 1 if item k supports item i and are in the same container

for i in range(n):
    for k in range(i+1,n):
        s_ik[i,k] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="s_ik[%s,%s]"%(i,k))

n_ik1 = {} # if left side of item k is less than or equal to item i

for i in range(n):
    for k in range(i+1,n):
        n_ik1[i,k] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="n_ik1[%s,%s]"%(i,k))

n_ik3 = {} # if right side of item i is less than or equal to item k

for i in range(n):
    for k in range(i+1,n):
        n_ik3[i,k] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="n_ik3[%s,%s]"%(i,k))

b_ikl = {} # if vertex l of item i is supported by item k

for i in range(n):
    for k in range(n):
        if i != k:
            for l in range(1,3):
                b_ikl[i,k,l] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="b_ikl[%s,%s,%s]"%(i,k,l))

v_ik = {} # represents |z_i_[k]-z_i[i]|

for i in range(n):
    for k in range(i+1,n):
        v_ik[i,k] = model.addVar(lb=0,ub=Hj_max, vtype=GRB.CONTINUOUS, name="v_ik[%s,%s]"%(i,k))

m_ik = {} # if z_i_[k] >= z_i[i]

for i in range(n):
    for k in range(i+1,n):
        m_ik[i,k] = model.addVar(lb=0,ub=1, vtype=GRB.BINARY, name="m_ik[%s,%s]"%(i,k))

model.update()     

Creating Decision Variables
--------------------


### Model Constraints

In [369]:
print('Creating Constraints')
print('--------------------')

# item i is constrained to single uld j (4)
for i in range(n):
    lhs = LinExpr()
    for j in range(m):
        lhs += p_ij[i,j]
    model.addConstr(lhs=lhs, sense=GRB.EQUAL, rhs=1, name='constraint_4')

# the horizontal position of each item i is constrained by the length of the container j (5)
for i in range(n):
    lhs = LinExpr()
    for j in range(m):    
        lhs += lc[j]*p_ij[i,j]
    model.addConstr(lhs=lhs, sense=GRB.GREATER_EQUAL, rhs=x_i_[i], name='constraint_5')

# the vertical position of each item i is constrained by the height of the container j (7)
for i in range(n):
    lhs = LinExpr()
    for j in range(m):
        lhs += hc[j]*p_ij[i,j]
    model.addConstr(lhs=lhs, sense=GRB.GREATER_EQUAL, rhs=z_i_[i], name='constraint_7')

# item length is length or height (8 & 10)
for i in range(n):
    lhs = LinExpr()
    lhs = x_i_[i]-x_i[i]
    model.addConstr(lhs=lhs, sense=GRB.EQUAL, rhs=r_i11[i]*li[i] + r_i13[i]*hi[i], name='constraint_8')
    lhs = LinExpr()
    lhs = z_i_[i]-z_i[i]
    model.addConstr(lhs=lhs, sense=GRB.EQUAL, rhs=r_i31[i]*li[i] + r_i33[i]*hi[i], name='constraint_10')

# side b of item i must only be along 1 axis a (11)
for i in range(n):
    for b in range(0,2):
        lhs = LinExpr()
        for a in range(0,2):
            lhs += r_iab[i,a,b]
        model.addConstr(lhs=lhs, sense=GRB.EQUAL, rhs=1, name='constraint_11')

# axis a must only be along 1 side b of item i (12)
for i in range(n):
    for a in range(0,2):
        lhs = LinExpr()
        for b in range(0,2):
            lhs += r_iab[i,a,b]
        model.addConstr(lhs=lhs, sense=GRB.EQUAL, rhs=1, name='constraint_12')   

## No Overlap Constraints ##
# ensure no overlap of items i in container j (13, 14, 15 & 18)

# only one item can be above and to the right
for i in range(n):
    for k in range(i+1,n):
        lhs = LinExpr()
        lhs = xp_ik[i,k]+xp_ki[k,i]+zp_ik[i,k]+zp_ki[k,i]
        for j in range(m):
            rhs = LinExpr()
            rhs = p_ij[i,j]+p_ij[k,j]-1
            model.addConstr(lhs=lhs, sense=GRB.GREATER_EQUAL, rhs=rhs, name='constraint_13')

for i in range(n): 
    for k in range(i+1,n): 
        lhs = LinExpr() 
        lhs += x_i_[k] 
        model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=x_i[i]+(1-xp_ik[i,k])*Lj_max, name='constraint_14') 

for i in range(n):
    for k in range(i+1,n):
        lhs = LinExpr()
        lhs += x_i[i]+1
        model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=x_i_[k]+xp_ik[i,k]*Lj_max, name='constraint_15')

for i in range(n):
    for k in range(i+1,n):
        lhs = LinExpr()
        lhs += z_i_[k]
        model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=z_i[i]+(1-zp_ik[i,k])*Hj_max, name='constraint_18')

## Orientation Constraints ##
# if length of box can be in a vertical orientation (19)
for i in range(n):
    lhs = LinExpr
    lhs = r_i31[i]
    model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=lv[i], name='constraint_19')

# if height of box can be in a vertical orientation (21)
for i in range(n):
    lhs = LinExpr
    lhs = r_i33[i]
    model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=hv[i], name='constraint_21')

## Stability Constraints ##
# item i must have 2 vertices supported by another item k if not supported by the ground (26)
for i in range(n):
    for k in range(n):
        if i != k:
            for l in range(1,3):
                lhs += 2*g_i[i] + b_ikl[i,k,l]

    model.addConstr(lhs=lhs, sense=GRB.EQUAL, rhs=2, name='constraint_26_modified')
    lhs = LinExpr()

# domain of lower vertex of item i z_i (27)
for i in range(n):
    lhs = LinExpr()
    lhs += z_i[i]
    model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=(1-g_i[i])*Hj_max, name='constraint_27')

# defining variable h_ik using v_ik where v_ik = |z_i_[k] - z_i[i]| (28-33)
for i in range(n):
    for k in range(i+1,n):
        lhs = LinExpr()
        lhs = z_i_[k]-z_i[i]
        model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=v_ik[i,k], name='constraint_28')
        lhs = LinExpr()
        lhs = z_i[i]-z_i_[k]
        model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=v_ik[i,k], name='constraint_29')
        lhs = LinExpr()
        lhs = v_ik[i,k]
        model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=z_i_[k]-z_i[i]+2*Hj_max*(1-m_ik[i,k]), name='constraint_30')
        model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=z_i[i]-z_i_[k]+2*Hj_max*m_ik[i,k], name='constraint_31')
        lhs = LinExpr()
        lhs = h_ik[i,k]
        model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=v_ik[i,k], name='constraint_32')
        lhs = LinExpr()
        lhs = v_ik[i,k]
        model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=h_ik[i,k]*Hj_max, name='constraint_33')

# if item i and k projections dont overlap then 1 <= 1 <= 2 (34 & 35)
for i in range(n):
    for k in range(i+1,n):
        lhs = LinExpr()
        lhs = o_ik[i,k]
        model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=xp_ik[i,k]+xp_ki[k,i], name='constraint_34')
        lhs = LinExpr()
        lhs = xp_ik[i,k]+xp_ki[k,i]
        model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=2*o_ik[i,k], name='constraint_34')

        lhs = LinExpr()
        lhs = 1-s_ik[i,k]
        model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=h_ik[i,k]+o_ik[i,k], name='constraint_35')
        lhs = LinExpr()
        lhs = h_ik[i,k]+o_ik[i,k]
        model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=2*(1-s_ik[i,k]), name='constraint_35')

# if k supports i then item i and k must be in the same container j (36 & 37)
for i in range(n):
    for k in range(i+1,n):
        for j in range(m):
            lhs = LinExpr()
            lhs = p_ij[i,j] - p_ij[k,j]
            model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=1-s_ik[i,k], name='constraint_36')
            lhs = LinExpr()
            lhs = p_ij[k,j] - p_ij[i,j]
            model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=1-s_ik[i,k], name='constraint_37')

# (38)
for i in range(n):
    for k in range(i+1,n):
        for l in range(1,3):
            lhs = LinExpr()
            lhs = b_ikl[i,k,l]
            model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=s_ik[i,k], name='constraint_38')

# (39-42) ???
for i in range(n):
    for k in range(i+1,n):
        lhs = LinExpr()
        lhs = n_ik1[i,k] + n_ik3[i,k]
        model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=2*(1-b_ikl[i,k,1]), name='constraint_39')
# (43 & 45)
for i in range(n):
    for k in range(i+1,n):
        lhs = LinExpr()
        lhs = x_i[k]
        model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=x_i[i]+n_ik1[i,k]*Lj_max, name='constraint_43')
        lhs = LinExpr()
        lhs = x_i_[i]
        model.addConstr(lhs=lhs, sense=GRB.LESS_EQUAL, rhs=x_i_[k]+n_ik3[i,k]*Lj_max, name='constraint_45')

# (49 & 50) ???

model.update()

Creating Constraints
--------------------


### Model Objective

In [370]:
# determine the total unused area by summing the unused area from each used container
for j in range(m):
        a_j[j] += lc[j]*hc[j]*u_j[j] - quicksum(li[i]*hi[i]*p_ij[i,j] for i in range(n))

# define objective
for j in range(m):
    model.setObjective(a_j[j], GRB.MINIMIZE)

model.update()

### Export Model

In [371]:
model.write('2D_Vertical_BPP.lp')

### Solve

In [372]:
model.setParam('MIPGap',0.001)
model.setParam('TimeLimit',2*3600)
model.params.LogFile='2D_BPP.log'

model.optimize()

solution = []

# Retrieve variable names and values
for v in model.getVars():
    solution.append([v.varName,v.x])

solution

Set parameter MIPGap to value 0.001
Set parameter TimeLimit to value 7200
Set parameter LogFile to value "2D_BPP.log"
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1210 rows, 849 columns and 3911 nonzeros
Model fingerprint: 0x47e32286
Variable types: 87 continuous, 762 integer (762 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 3e+04]
  Bounds range     [1e+00, 5e+04]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -2697.000000
Presolve removed 748 rows and 557 columns
Presolve time: 0.01s
Presolved: 462 rows, 292 columns, 1373 nonzeros
Variable types: 83 continuous, 209 integer (209 binary)
Found heuristic solution: objective -36689.00000

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.01 work units)
Thread count

[['p[0,0]', 0.0],
 ['p[0,1]', 1.0],
 ['p[1,0]', 0.0],
 ['p[1,1]', 1.0],
 ['p[2,0]', 0.0],
 ['p[2,1]', 1.0],
 ['p[3,0]', 0.0],
 ['p[3,1]', 1.0],
 ['p[4,0]', 0.0],
 ['p[4,1]', 1.0],
 ['p[5,0]', 0.0],
 ['p[5,1]', 1.0],
 ['p[6,0]', 0.0],
 ['p[6,1]', 1.0],
 ['p[7,0]', 0.0],
 ['p[7,1]', 1.0],
 ['p[8,0]', 0.0],
 ['p[8,1]', 1.0],
 ['p[9,0]', 0.0],
 ['p[9,1]', 1.0],
 ['u[0]', 0.0],
 ['a[0]', 0.0],
 ['u[1]', 0.0],
 ['a[1]', 0.0],
 ['x_[0]', 0.0],
 ['x_[0]_', 0.0],
 ['z_[0]', 0.0],
 ['z_[0]_', 0.0],
 ['x_[1]', 0.0],
 ['x_[1]_', 0.0],
 ['z_[1]', 0.0],
 ['z_[1]_', 0.0],
 ['x_[2]', 1.0],
 ['x_[2]_', 1.0],
 ['z_[2]', 0.0],
 ['z_[2]_', 0.0],
 ['x_[3]', 0.0],
 ['x_[3]_', 0.0],
 ['z_[3]', 0.0],
 ['z_[3]_', 0.0],
 ['x_[4]', 0.0],
 ['x_[4]_', 0.0],
 ['z_[4]', 0.0],
 ['z_[4]_', 0.0],
 ['x_[5]', 0.0],
 ['x_[5]_', 0.0],
 ['z_[5]', 0.0],
 ['z_[5]_', 0.0],
 ['x_[6]', 1.0],
 ['x_[6]_', 1.0],
 ['z_[6]', 0.0],
 ['z_[6]_', 0.0],
 ['x_[7]', 0.0],
 ['x_[7]_', 0.0],
 ['z_[7]', 0.0],
 ['z_[7]_', 0.0],
 ['x_[8]', 1.0],